In [1]:
# data science
import pandas as pd
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func, MetaData, Table, Column, Integer, String, Float, BigInteger

In [2]:
df = pd.read_csv('backup_data/onlinefoods.csv')

print(df.info())
df. head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         388 non-null    int64  
 1   Gender                      388 non-null    object 
 2   Marital Status              388 non-null    object 
 3   Occupation                  388 non-null    object 
 4   Monthly Income              388 non-null    object 
 5   Educational Qualifications  388 non-null    object 
 6   Family size                 388 non-null    int64  
 7   latitude                    388 non-null    float64
 8   longitude                   388 non-null    float64
 9   Pin code                    388 non-null    int64  
 10  Output                      388 non-null    object 
 11  Feedback                    388 non-null    object 
 12  Unnamed: 12                 388 non-null    object 
dtypes: float64(2), int64(3), object(8)


,Age,Gender,Marital Status,Occupation,Monthly Income,Educational Qualifications,Family size,latitude,longitude,Pin code,Output,Feedback,Unnamed: 12
0,20,Female,Single,Student,No Income,Post Graduate,4,12.9766,77.5993,560001,Yes,Positive,Yes
1,24,Female,Single,Student,Below Rs.10000,Graduate,3,12.9770,77.5773,560009,Yes,Positive,Yes
2,22,Male,Single,Student,Below Rs.10000,Post Graduate,3,12.9551,77.6593,560017,Yes,Negative,Yes
3,22,Female,Single,Student,No Income,Graduate,6,12.9473,77.5616,560019,Yes,Positive,Yes
4,22,Male,Single,Student,Below Rs.10000,Post Graduate,4,12.9850,77.5533,560010,Yes,Positive,Yes


In [3]:
df = df.loc[:,["Age", "Gender", "Marital Status", "Occupation", "Monthly Income", "Educational Qualifications", "Family size", "latitude", "longitude", "Pin code"]]

print(df.info())
df. head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         388 non-null    int64  
 1   Gender                      388 non-null    object 
 2   Marital Status              388 non-null    object 
 3   Occupation                  388 non-null    object 
 4   Monthly Income              388 non-null    object 
 5   Educational Qualifications  388 non-null    object 
 6   Family size                 388 non-null    int64  
 7   latitude                    388 non-null    float64
 8   longitude                   388 non-null    float64
 9   Pin code                    388 non-null    int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 30.4+ KB
None


,Age,Gender,Marital Status,Occupation,Monthly Income,Educational Qualifications,Family size,latitude,longitude,Pin code
0,20,Female,Single,Student,No Income,Post Graduate,4,12.9766,77.5993,560001
1,24,Female,Single,Student,Below Rs.10000,Graduate,3,12.9770,77.5773,560009
2,22,Male,Single,Student,Below Rs.10000,Post Graduate,3,12.9551,77.6593,560017
3,22,Female,Single,Student,No Income,Graduate,6,12.9473,77.5616,560019
4,22,Male,Single,Student,Below Rs.10000,Post Graduate,4,12.9850,77.5533,560010


In [4]:
df["age"] = df["Age"]
df["gender"] = df["Gender"]
df["marital_status"] = df["Marital Status"]
df["occupation"] = df["Occupation"]
df["monthly_income"] = df["Monthly Income"]
df["educational_qualifications"] = df["Educational Qualifications"]
df["family_size"] = df["Family size"]
df["pin_code"] = df["Pin code"]
df.drop(["Age", "Gender", "Marital Status", "Occupation", "Monthly Income", "Educational Qualifications", "Family size", "Pin code"], axis=1, inplace=True)

df.head()

,latitude,longitude,age,gender,marital_status,occupation,monthly_income,educational_qualifications,family_size,pin_code
0,12.9766,77.5993,20,Female,Single,Student,No Income,Post Graduate,4,560001
1,12.9770,77.5773,24,Female,Single,Student,Below Rs.10000,Graduate,3,560009
2,12.9551,77.6593,22,Male,Single,Student,Below Rs.10000,Post Graduate,3,560017
3,12.9473,77.5616,22,Female,Single,Student,No Income,Graduate,6,560019
4,12.9850,77.5533,22,Male,Single,Student,Below Rs.10000,Post Graduate,4,560010


In [5]:
engine = create_engine("sqlite:///onlinefoods.sqlite")

In [6]:
meta = MetaData()

onlinefoods = Table(
    "onlinefoods", meta,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("age", Integer),
    Column("gender", String),
    Column("marital_status", String),
    Column("occupation", String),
    Column("monthly_income", String),
    Column("educational_qualifications", String),
    Column("family_size", Integer),
    Column("latitude", Float),
    Column("longitude", Float),
    Column("pin_code", Integer),
)

meta.create_all(engine)

df.to_sql('onlinefoods', con=engine, if_exists='append', index=False)

388

In [7]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

onlinefoods
-----------
id INTEGER
age INTEGER
gender VARCHAR
marital_status VARCHAR
occupation VARCHAR
monthly_income VARCHAR
educational_qualifications VARCHAR
family_size INTEGER
latitude FLOAT
longitude FLOAT
pin_code INTEGER



In [8]:
session = Session(bind=engine)

print(session.query(onlinefoods).first())
session.close()

(1, 20, 'Female', 'Single', 'Student', 'No Income', 'Post Graduate', 4, 12.9766, 77.5993, 560001)


In [9]:
engine.dispose()